In [ ]:
import os
import random
import blocksci
import copy
import numpy as np
import pickle
import tqdm

chain = blocksci.Blockchain("/home/blocksci_data/blocksci.conf")

random.seed(42)


In [ ]:


def hasStr(address):
    try:
        string = address.address_string
        return True
    except:
        return False

    
def feature(address):
    temp_data = []
    for func in functions_addr:
        rs = func(address)
        temp_data.extend(rs if isinstance(rs, tuple) else [rs])
    return temp_data
    
    
    
def generate_graph(address_target, n=3, limit=100000):
    i2a = {0:address_target}
    a2i = {address_target:0}
    edges = [[],[]] #src dst
    ndata = []
    
    eset = set()
    ring = set() 
    ring.add(address_target)
    feat = feature(chain.address_from_string(address_target))
    ndata.append(copy.copy(feat))
    n_tqdm = tqdm.tqdm(total=n, desc=address_target, mininterval=10)
    for nt in range(n):
        ring2 = set()
        for src in ring:
            address = chain.address_from_string(src)
            cnt = 0
            for in_tx in address.input_txes:
                for out in in_tx.outputs:
                    if len(a2i) >= limit or cnt >= 1000:
                        break
                    if not hasStr(out.address) or str(out.address.address_string)==src:
                        continue
                    dst = out.address.address_string
                    if dst in a2i:
                        if (a2i[src], a2i[dst]) not in eset:
                            edges[0].append(a2i[src])
                            edges[1].append(a2i[dst])
                            eset.add((a2i[src], a2i[dst]))
                        continue
                    a2i[dst] = len(a2i)
                    i2a[a2i[dst]] = dst
                    if (a2i[src], a2i[dst]) not in eset:
                        edges[0].append(a2i[src])
                        edges[1].append(a2i[dst])
                        eset.add((a2i[src], a2i[dst]))
                    feat = feature(out.address)
                    ndata.append(copy.copy(feat))
                    ring2.add(dst)
                    n_tqdm.set_postfix(addr=src, subgraph=len(a2i), next_ring=len(ring2), step=1, hop=nt+1)
                    cnt += 1
                if len(a2i) >= limit or cnt >= 1000:
                    break
            if len(a2i) >= limit or cnt >= 1000:
                    break
            for out_tx in address.output_txes:
                for _in in out_tx.inputs:
                    if len(a2i) >= limit or cnt >= 1000:
                        break
                    if not hasStr(_in.address) or str(_in.address.address_string)==src:
                        continue
                    dst = _in.address.address_string
                    if _in.address.address_string in a2i:
                        if (a2i[src], a2i[dst]) not in eset:
                            edges[0].append(a2i[src])
                            edges[1].append(a2i[dst])
                            eset.add((a2i[src], a2i[dst]))
                        continue
                    a2i[dst] = len(a2i)
                    i2a[a2i[dst]] = dst
                    if (a2i[src], a2i[dst]) not in eset:
                        edges[0].append(a2i[src])
                        edges[1].append(a2i[dst])
                        eset.add((a2i[src], a2i[dst]))
                    feat = feature(_in.address)
                    ndata.append(copy.copy(feat))
                    ring2.add(dst)
                    n_tqdm.set_postfix(addr=src, subgraph=len(a2i), next_ring=len(ring2), step=2, hop=nt+1)        
                if len(a2i) >= limit or cnt >= 1000:
                    break
            if len(a2i) >= limit:
                break 
        ring = ring2
        if len(a2i) >= limit:
            n_tqdm.set_postfix(subgraph=len(a2i), next_ring=0, step=3, hop=nt+1)
            break
        n_tqdm.update(1)
    n_tqdm.set_postfix(subgraph=len(a2i), next_ring=0, step=3, hop=nt+1)
    for src in ring:
        address = chain.address_from_string(src)
        n_tqdm.set_postfix(addr=src, subgraph=len(a2i), next_ring=0, step=3, hop=nt+1)
        for in_tx in address.input_txes:
            for out in in_tx.outputs:
                if hasStr(out.address) and out.address.address_string in a2i and str(out.address.address_string)!=src and (a2i[src], a2i[out.address.address_string]) not in eset:
                    edges[0].append(a2i[src])
                    edges[1].append(a2i[str(out.address.address_string)])
                    eset.add((a2i[src], a2i[out.address.address_string]))
        for out_tx in address.output_txes:
            for _in in out_tx.inputs:
                if hasStr(_in.address) and _in.address.address_string in a2i and str(_in.address.address_string)!=src and (a2i[src], a2i[_in.address.address_string]) not in eset:
                    edges[0].append(a2i[src])
                    edges[1].append(a2i[str(_in.address.address_string)])   
                    eset.add((a2i[src], a2i[_in.address.address_string]))
    return a2i, i2a, edges, ndata
        

files = os.listdir('addresses')

for file in files:
    address_type = file[:-4]
    if not os.path.exists('graph_data/{}'.format(address_type)):
        os.mkdir('graph_data/{}'.format(address_type))
    with open('addresses/'+file, 'r') as f:
        addresses = f.readlines()
        for address in addresses:
            address = address.strip()
            dataDir = 'graph_data/{}/{}'.format(address_type, address)
            if not os.path.exists(dataDir):
                os.mkdir(dataDir)
            a2i, i2a, es, ndata = generate_graph(address)

            output = open(dataDir+'/a2i.pkl', 'wb')
            pickle.dump(a2i, output)
            output.close()
            output = open(dataDir+'/i2a.pkl', 'wb')
            pickle.dump(i2a, output)
            output.close()
            output = open(dataDir+'/es.pkl', 'wb')
            pickle.dump(es, output)
            output.close()
            output = open(dataDir+'/ndata.pkl', 'wb')
            pickle.dump(ndata, output)
            output.close()


In [ ]:
functions_addr = []


def sign_up_decorator(_functions_list: list):
    def sign_up(module_func):
        _functions_list.append(module_func)
        def run_module(*args):
            return module_func(*args)
        return run_module
    return sign_up


'''--------------------------------------------------------amount--------------------------------------------------------'''
@sign_up_decorator(functions_addr)
def basic_module_311(address):
    return len([tx for tx in address.txes if tx.block_height])

@sign_up_decorator(functions_addr)
def basic_module_312(address):
    ins = [_input for _input in address.inputs if _input.block.height]
    outs = [_output for _output in address.outputs if _output.block.height]
    return len(ins), \
           len(outs)

@sign_up_decorator(functions_addr)
def basic_module_313(address):
    txes = [tx for tx in address.txes if tx.block_height]
    ins = [tx.input_count for tx in txes]
    outs = [tx.output_count for tx in txes]
    return sum(ins) / len(ins), max(ins), min(ins),\
           sum(outs) / len(outs), max(outs), min(outs)

@sign_up_decorator(functions_addr)
def basic_module_314(address):
    ins_n, outs_n = basic_module_312(address)
    return ins_n / outs_n

@sign_up_decorator(functions_addr)
def basic_module_315(address):
    txes = [tx for tx in address.txes if tx.block_height]
    values = [tx.input_count / tx.output_count for tx in txes]
    return sum(values) / len(values), max(values), min(values)

@sign_up_decorator(functions_addr)
def basic_module_316(address):
    return sum([_input.value for _input in address.inputs if _input.block.height]), \
           sum([_output.value for _output in address.outputs if _output.block.height])

@sign_up_decorator(functions_addr)
def basic_module_317(address):
    txes = [tx for tx in address.txes if tx.block_height]
    in_values = [tx.input_value for tx in txes]
    out_values = [tx.output_value for tx in txes]
    return sum(in_values) / len(in_values), max(in_values), min(in_values),\
           sum(out_values) / len(out_values), max(out_values), min(out_values)

@sign_up_decorator(functions_addr)
def basic_module_318(address):
    txes = [tx for tx in address.txes if tx.block_height]
    values = [tx.fee for tx in txes]
    return sum(values) / len(values), max(values), min(values)

@sign_up_decorator(functions_addr)
def basic_module_319(address):
    return address.balance()

'''--------------------------------------------------------time--------------------------------------------------------'''
@sign_up_decorator(functions_addr)
def basic_module_321(address):
    txes = [tx for tx in address.txes if tx.block_height]
    active_n = set([tx.block_height for tx in txes])
    return len(active_n)

@sign_up_decorator(functions_addr)
def basic_module_322(address):
    txes = [tx for tx in address.txes if tx.block_height]
    tx_count = {}
    for tx in txes:
        if tx.block_height in tx_count:
            tx_count[tx.block_height] += 1
        else:
            tx_count[tx.block_height] = 0
    tx_count = list(tx_count.values())
    return sum(tx_count) / len(tx_count), max(tx_count), min(tx_count)
    
@sign_up_decorator(functions_addr)
def basic_module_323(address):
    txes = [tx for tx in address.txes if tx.block_height]
    active_n = sorted(list(set([tx.block_height for tx in txes])))
    if len(active_n) <= 1:
        return 0, 0, 0
    else:
        for index in range(len(active_n) - 1):
            active_n[index] = abs(active_n[index + 1] - active_n[index])
        active_HD = active_n[:-1]
        return sum(active_HD) / len(active_HD), max(active_HD), min(active_HD)
    
@sign_up_decorator(functions_addr)
def basic_module_324(address):
    txes = [tx for tx in address.txes if tx.block_height]
    active_max_HD = sorted(list(set([tx.block_height for tx in txes])))
    return max(active_max_HD) - min(active_max_HD)

@sign_up_decorator(functions_addr)
def basic_module_325(address):
    ins_HD = [_input.age for _input in address.inputs if _input.block.height]
    if ins_HD:
        return sum(ins_HD) / max(len(ins_HD), 1), max(ins_HD), min(ins_HD)
    else:
         return 0, 0, 0


@sign_up_decorator(functions_addr)
def basic_module_326(address):
    outs_HD = [_output.spending_input.age for _output in address.outputs if _output.block.height and _output.is_spent]
    if len(outs_HD):
        return sum(outs_HD) / len(outs_HD), max(outs_HD), min(outs_HD)
    else:
        return 0, 0, 0

'''--------------------------------------------------------amount+time--------------------------------------------------------'''
@sign_up_decorator(functions_addr)
def basic_module_331(address):
    tx_n = basic_module_311(address)
    active_n = basic_module_321(address)
    max_HD = max(basic_module_324(address), 1)
    return tx_n / active_n, tx_n / max_HD

@sign_up_decorator(functions_addr)
def basic_module_332(address):
    ins_n, outs_n = basic_module_312(address)
    active_n = basic_module_321(address)
    max_HD = max(basic_module_324(address), 1)
    return ins_n / active_n, outs_n / active_n, ins_n / max_HD, outs_n / max_HD

@sign_up_decorator(functions_addr)
def basic_module_333(address):
    ins_value, outs_value = basic_module_316(address)
    active_n = basic_module_321(address)
    max_HD = max(basic_module_324(address), 1)
    return ins_value / active_n, outs_value / active_n, ins_value / max_HD, outs_value / max_HD